In [1]:
import numpy as np

from HMM import *
from toolbox import *

In [2]:
# Import and separate datasets
ERROR_RATE = 10  # 10% or 20%
train_set, test_set = load_db(error_rate=ERROR_RATE)
X_train = [[token[0] for token in word] for word in train_set]
y_train = [[token[1] for token in word] for word in train_set]
X_test = [[token[0] for token in word] for word in test_set]
y_test = [[token[1] for token in word] for word in test_set]

# Get states and observations sets
states, observations = get_observations_states(X_train, y_train)
print("{} states :\n{}".format(len(states), states))
print("{} observations :\n{}".format(len(observations), observations))

# Example from dataset
print("\nSample example (observation, état) :\n{}".format(train_set[3]))

26 states :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
26 observations :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

Sample example (observation, état) :
[('a', 'a'), ('c', 'c'), ('v', 'c'), ('o', 'o'), ('u', 'u'), ('n', 'n'), ('t', 't')]


In [3]:
# Initialize and train HMM
hmm1 = HMM(states, observations)
hmm1.EM(X_train)

1st order HMM created with: 
 * 26 states
 * 26 observations
EM LOOP: n_iter= 0
EM LOOP: n_iter= 1
EM LOOP: n_iter= 2
EM LOOP: n_iter= 3
EM LOOP: n_iter= 4
EM LOOP: n_iter= 5
EM LOOP: n_iter= 6
EM LOOP: n_iter= 7
EM LOOP: n_iter= 8
EM LOOP: n_iter= 9
EM LOOP: n_iter= 10
EM LOOP: n_iter= 11
EM LOOP: n_iter= 12
EM LOOP: n_iter= 13
EM LOOP: n_iter= 14
EM LOOP: n_iter= 15
EM LOOP: n_iter= 16
EM LOOP: n_iter= 17
EM LOOP: n_iter= 18
EM LOOP: n_iter= 19
EM LOOP: n_iter= 20
EM LOOP: n_iter= 21
EM LOOP: n_iter= 22
EM LOOP: n_iter= 23
EM LOOP: n_iter= 24
EM LOOP: n_iter= 25


In [4]:
y_test_pred = hmm1.predict(X_test)
display_correction_stats(X_test, y_test, y_test_pred, name="HMM1")

HMM1 score on test set
 * accuracy on full words : 2.53%
 * accuracy on letters    : 7.69%
   > typos corrected      : 56 (0.77%)
   > typos not corrected  : 689 (9.41%)
   > typos added          : 6068 (82.90%)

Dummy score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 89.82%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 745 (10.18%)
   > typos added          : 0 (0.00%)
